In [1]:
import requests
import pandas as pd

# Replace with your actual API key
API_KEY = "391d2c632b433f4ed3a318ad2bdb814e"
SPORT = "basketball_ncaab"  # Sport key for NCAAB
REGION = "us"  # Odds for the US market
#BOOKMAKERS = ["draftkings", "fanduel", "espn"]  # Add/remove sportsbooks

# API URL
URL = f"https://api.the-odds-api.com/v4/sports/{SPORT}/odds/"

# Request Parameters
params = {
    "api_key": API_KEY,
    "regions": REGION,
    "markets": "spreads" #"h2h,spreads,totals"  # Moneyline, spreads, and over/under
    #"bookmakers": ",".join(BOOKMAKERS),
}

# Make API request
response = requests.get(URL, params=params)

if response.status_code == 200:
    odds_data = response.json()

    # Convert to DataFrame
    games = []
    for game in odds_data:
        for bookmaker in game.get("bookmakers", []):
            for market in bookmaker.get("markets", []):
                for outcome in market.get("outcomes", []):
                    games.append({
                        "Game": f"{game['home_team']} vs {game['away_team']}",
                        "Bookmaker": bookmaker["key"],
                        "Market": market["key"],
                        "Team": outcome.get("name"),
                        "Odds": outcome.get("price"),
                        "Point": outcome.get("point")  # For spreads/totals
                    })

    df = pd.DataFrame(games)
    print(df.head())  # Show the first few rows

else:
    print(f"Error: {response.status_code}, {response.text}")


                                       Game   Bookmaker   Market  \
0  Michigan St Spartans vs Duke Blue Devils     fanduel  spreads   
1  Michigan St Spartans vs Duke Blue Devils     fanduel  spreads   
2  Michigan St Spartans vs Duke Blue Devils  draftkings  spreads   
3  Michigan St Spartans vs Duke Blue Devils  draftkings  spreads   
4  Michigan St Spartans vs Duke Blue Devils   betrivers  spreads   

                   Team  Odds  Point  
0      Duke Blue Devils  2.00   -1.5  
1  Michigan St Spartans  1.82    1.5  
2      Duke Blue Devils  2.02   -1.5  
3  Michigan St Spartans  1.82    1.5  
4      Duke Blue Devils  1.93   -1.0  


In [2]:
print(odds_data)

[{'id': 'a7f71171dcb62609a3a6f2df23a28632', 'sport_key': 'basketball_ncaab', 'sport_title': 'NCAAB', 'commence_time': '2025-12-06T16:59:00Z', 'home_team': 'Michigan St Spartans', 'away_team': 'Duke Blue Devils', 'bookmakers': [{'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2025-12-06T16:20:14Z', 'markets': [{'key': 'spreads', 'last_update': '2025-12-06T16:20:14Z', 'outcomes': [{'name': 'Duke Blue Devils', 'price': 2.0, 'point': -1.5}, {'name': 'Michigan St Spartans', 'price': 1.82, 'point': 1.5}]}]}, {'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2025-12-06T16:20:31Z', 'markets': [{'key': 'spreads', 'last_update': '2025-12-06T16:20:30Z', 'outcomes': [{'name': 'Duke Blue Devils', 'price': 2.02, 'point': -1.5}, {'name': 'Michigan St Spartans', 'price': 1.82, 'point': 1.5}]}]}, {'key': 'betrivers', 'title': 'BetRivers', 'last_update': '2025-12-06T16:19:55Z', 'markets': [{'key': 'spreads', 'last_update': '2025-12-06T16:19:54Z', 'outcomes': [{'name': 'Duke Blue Devils'

In [3]:
df.to_csv('odds.csv', index=False)

In [4]:
odds = pd.read_csv('odds.csv')

In [5]:
odds['Team'].unique()

array(['Duke Blue Devils', 'Michigan St Spartans', 'Bucknell Bison',
       'UMBC Retrievers', 'Dayton Flyers', 'Virginia Cavaliers',
       'Georgia Tech Yellow Jackets', 'Monmouth Hawks',
       'Iowa State Cyclones', 'Purdue Boilermakers', 'Miami Hurricanes',
       'Southern Miss Golden Eagles', 'Providence Friars',
       'Rhode Island Rams', 'Boston College Eagles', 'New Haven Chargers',
       'Old Dominion Monarchs', 'Richmond Spiders',
       'Bowling Green Falcons', 'Utah Valley Wolverines',
       'Maine Black Bears', 'Miami (OH) RedHawks', 'NC State Wolfpack',
       'UNC Asheville Bulldogs', 'American Eagles',
       'Maryland-Eastern Shore Hawks', 'Boise State Broncos',
       'Butler Bulldogs', 'Buffalo Bulls', 'St. Bonaventure Bonnies',
       'Cleveland St Vikings', 'Detroit Mercy Titans',
       'Coastal Carolina Chanticleers', 'Winthrop Eagles',
       'Coppin St Eagles', 'Liberty Flames', 'Delaware Blue Hens',
       'Delaware St Hornets', 'Duquesne Dukes', 'Stony B

In [6]:
#Create list of teams with two names
teams = ['Southern Miss Golden Eagles','Maine Black Bears', 'UMass Lowell River Hawks', 'Albany Great Danes', 'Georgia Tech Yellow Jackets', 'Oakland Golden Grizzlies', 'Kent State Golden Flashes'
         ,'Lehigh Mountain Hawks','North Carolina Tar Heels','Rutgers Scarlet Knights','TCU Horned Frogs','Tulsa Golden Hurricane',"Louisiana Ragin' Cajuns",'Nevada Wolf Pack','Arizona St Sun Devils',
         'DePaul Blue Demons','Alabama Crimson Tide','Central Connecticut St Blue Devils','Tennessee Tech Golden Eagles','Texas Tech Red Raiders','Oral Roberts Golden Eagles'
         ,'St. Francis (PA) Red Flash','Marquette Golden Eagles','Minnesota Golden Gophers','California Golden Bears','Notre Dame Fighting Irish','Illinois Fighting Illini','Texas Tech Red Raiders'
         ,'Duke Blue Devils']

In [7]:
# Function to remove the last word
def remove_last_word(text):
    if text == 'Over' or text == 'Under':
        return text
    if text in teams:
        return ' '.join(text.split()[:-2])
    else:
        return ' '.join(text.split()[:-1])

In [8]:
def convert_decimal_to_american(decimal_odds):
    if decimal_odds < 2:
        return -100 / (decimal_odds - 1)
    else:
        return decimal_odds - 1

In [9]:
# Apply the function to the DataFrame column
odds['Team Modified'] = odds['Team'].apply(remove_last_word)

In [10]:
odds['Team Modified'] = odds['Team Modified'].str.replace('State', 'St')
odds['Team Modified'] = odds['Team Modified'].str.replace('St', 'St.')

In [48]:
odds['Implied Probability'] = 1 / odds['Odds']

#Set vegas line column by converting decimal odds to american odds
odds['Vegas Line'] = odds['Odds'].apply(convert_decimal_to_american)

In [50]:
final_odds = odds[['Team Modified', 'Market', 'Bookmaker', 'Point', 'Implied Probability', 'Vegas Line']]
final_odds.to_csv('12-03-2025 Odds.csv', index=False) 

In [14]:
final_odds['Bookmaker'].unique()

array(['fanduel', 'betonlineag', 'lowvig', 'mybookieag', 'draftkings',
       'betmgm', 'betrivers', 'betus', 'bovada'], dtype=object)

In [12]:
final_odds

,Team Modified,Market,Bookmaker,Point,Implied Probability,Vegas Line
0,Rhode Island,spreads,draftkings,1.5,0.534759,-114.942529
1,Towson,spreads,draftkings,-1.5,0.534759,-114.942529
2,Rhode Island,spreads,betrivers,1.5,0.584795,-140.845070
3,Towson,spreads,betrivers,-1.5,0.515464,-106.382979
4,Rhode Island,spreads,betmgm,1.5,0.512821,-105.263158
...,...,...,...,...,...,...
857,Maryland,spreads,draftkings,13.5,0.523560,-109.890110
858,Oregon,spreads,draftkings,-1.5,0.523560,-109.890110
859,San Diego St.,spreads,draftkings,1.5,0.523560,-109.890110
860,Alabama,spreads,draftkings,-13.5,0.523560,-109.890110
